In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd

#GPU 사용
device = torch.device("cuda:0")

     |████████████████████████████████| 46.9 MB 42 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=b66c44dfb3f7449f12440641a7086b93d415d254ebd9b15879afbac0d8a87eea
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.0 MB/s 
     |████████████████████████████████| 754 kB 7.1 MB/s 
     |████████████████████████████████| 3.0 MB 42.2 MB/s 
     |████████████████████████████████| 895 kB 68.6 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-0nj8r0a1
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip

In [ ]:
from array import array
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')


# 원래 데이터
PATH = '/content/drive/MyDrive/Colab Notebooks/KUBIG_NLP/'
train_data = pd.read_csv(PATH + "train_data.csv")
test_data = pd.read_csv(PATH + "test_data.csv")
topic_dict = pd.read_csv(PATH + "topic_dict.csv")
submission = pd.read_csv(PATH + "sample_submission.csv")

# 은지님 전처리 데이터
# X_test = np.load(PATH+'X_test.npy')
# y_test= np.load(PATH+'y_test.npy')

# X_train = np.load(PATH+'X_train.npy')
# y_train = np.load(PATH+'y_train.npy')

Mounted at /content/drive


In [ ]:
train_dataset = []
for sen, label in zip(train_data['title'], train_data['topic_idx']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], )) # 시간 오래걸려서 조정 보통 3~5 사용 3으로 했을때 84까지 나옴

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 4 
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##7개 카테고리로 분류
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.4).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    # test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8272252082824707 train acc 0.25
epoch 1 batch id 201 loss 1.0595110654830933 train acc 0.41340174129353235
epoch 1 batch id 401 loss 0.9282589554786682 train acc 0.6150249376558603
epoch 1 batch id 601 loss 0.6683393716812134 train acc 0.6853161397670549
epoch 1 batch id 801 loss 0.36948174238204956 train acc 0.7239778401997503
epoch 1 batch id 1001 loss 0.0909237489104271 train acc 0.7645791708291708
epoch 1 batch id 1201 loss 0.4267745018005371 train acc 0.7906692339716903
epoch 1 batch id 1401 loss 0.17805814743041992 train acc 0.7990943968593861
epoch 1 train acc 0.8017037491240364


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.7753239274024963 train acc 0.78125
epoch 2 batch id 201 loss 0.42940106987953186 train acc 0.8726679104477612
epoch 2 batch id 401 loss 0.5698414444923401 train acc 0.8640118453865336
epoch 2 batch id 601 loss 0.4470161199569702 train acc 0.8669925124792013
epoch 2 batch id 801 loss 0.21314606070518494 train acc 0.8705134207240949
epoch 2 batch id 1001 loss 0.011688560247421265 train acc 0.8867694805194806
epoch 2 batch id 1201 loss 0.21635626256465912 train acc 0.8973251457119067
epoch 2 batch id 1401 loss 0.14534544944763184 train acc 0.898108493932905
epoch 2 train acc 0.8993517869656622


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 3 batch id 1 loss 0.3640446364879608 train acc 0.875
epoch 3 batch id 201 loss 0.36298996210098267 train acc 0.9102922885572139
epoch 3 batch id 401 loss 0.39073535799980164 train acc 0.9051589775561097
epoch 3 batch id 601 loss 0.22181721031665802 train acc 0.9074979201331115
epoch 3 batch id 801 loss 0.21146079897880554 train acc 0.9102684144818977
epoch 3 batch id 1001 loss 0.004676952958106995 train acc 0.9230144855144855
epoch 3 batch id 1201 loss 0.19338008761405945 train acc 0.9311771440466278
epoch 3 batch id 1401 loss 0.02673988603055477 train acc 0.9320128479657388
epoch 3 train acc 0.9329668885774351


  0%|          | 0/1427 [00:00<?, ?it/s]

epoch 4 batch id 1 loss 0.43105894327163696 train acc 0.90625
epoch 4 batch id 201 loss 0.25928354263305664 train acc 0.9357898009950248
epoch 4 batch id 401 loss 0.2646643817424774 train acc 0.9343827930174564
epoch 4 batch id 601 loss 0.2471175491809845 train acc 0.9364600665557404
epoch 4 batch id 801 loss 0.11763133853673935 train acc 0.9386704119850188
epoch 4 batch id 1001 loss 0.022024845704436302 train acc 0.9473963536463537
epoch 4 batch id 1201 loss 0.08862394839525223 train acc 0.9526436303080766
epoch 4 batch id 1401 loss 0.011783912777900696 train acc 0.9531807637401856
epoch 4 train acc 0.9538586194814296


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(sentence):
  data = [sentence, '0']
  dataset_another = [data]
  logits = 0
  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
  
  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)

      valid_length= valid_length
      label = label.long().to(device)

      out = model(token_ids, valid_length, segment_ids)

      for i in out:
        logits = i
        logits = logits.detach().cpu().numpy()
        logits = np.argmax(logits)

  return logits

using cached model


In [ ]:
label_answer = []
step = 0
for i in tqdm(test_data['title']):
  label_answer.append(predict(i))

  0%|          | 0/9131 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 9131/9131 [1:44:53<00:00,  1.45it/s]


In [ ]:
submission['topic_idx'] = label_answer
submission.to_csv(PATH + 'private.csv', index=False)